# Inference with uncertainty bounds

The last two notebooks explained upward and downward inference. In this notebook we will show inference with uncertainty bounds.

## Motivation
The main motivation is that not every time we know precisely if our data point can be trusted. In that case we would like to do inference which take this into account.

## Grecian domain

We will use same domain as for downward inference notebook with small change. Let us recap it.

Description in natural language is as follows:

* All grecian is a native or resident of Greece.<br>
* Grecians are mortal.<br>
* **We are not sure if Zeus is native or resident of Greece, is not mortal.**<br>
* Socrates is native of Greece (is not resident of Greece), is mortal.<br>
* Confucius is neither native or resident of Greece, is mortal.<br>
* Great Pyramid of Giza is neither native or resident of Greece, is not mortal.

Note we added the uncertainity to our natural language description. The origin of Zeus is unknown for us.

### First order logic description
Now let us translate this domain into First-order logic.

#### Predicates
Natural language definition gives us four predicates: $grecian(x)$, $mortal(x)$, $native\_of\_greece(x)$ and $resident\_of\_greece(x)$. See previous notebooks for detailed discussion.

#### Formulae
The sentence "All grecian is a native or resident of Greece" can be formulated as following axiom:

$(\forall x) (grecian(x) \Leftrightarrow native\_of\_greece(x) \lor resident\_of\_greece(x))$

The sentence "Grecians are mortal" is simple formula with logical implication:

$grecian(x) \implies mortal(x)$

#### Objects
Our natural language description includes following set of objects: `{Zeus, Socrates, Confucius, Great Pyramid of Giza}`. Their relations are represented by following table. Note that we have question marks in row with Zeus.

![grecian_downward_tt_unknown](images/grecian_downward_tt_unknown.png)

### Our goal
Since we are not sure about the origin of Zeus we would like to do inference which takes this uncertainty into account. What does it mean? We expect that downward inference will not be able to infer if Zeus is grecian or not.

Situation is described by following table:

![grecian_downward_tt](images/grecian_downward_tt_unknown_overview.png)

### Definition of LNN model

Now we will define our model in LNN library. The definition is almost same as in the previous two notebooks.

First we will create an empty model.

In [13]:
from lnn import Model

model = Model()

Define our predicates and add them to the model.

In [14]:
from lnn import Predicate

grecian = Predicate(name="grecian")
mortal = Predicate(name="mortal")
native_of_greece = Predicate(name="native_of_greece")
resident_of_greece = Predicate(name="resident_of_greece")

model.add_formulae(grecian, mortal, native_of_greece, resident_of_greece)

Now we define variable and rest of the formulae than add them to the model.

In [15]:
from lnn import Variable, Implies, World, ForAll, Or, Bidirectional

x = Variable("x")

all_grecians_are_native_or_resident = ForAll(
    x,
    Bidirectional(
        Or(native_of_greece(x), resident_of_greece(x), name="native_or_resident"),
        grecian(x),
        name="grecians_are_native_or_resident",
    ),
    name="all_grecians_are_native_or_resident",
    world=World.AXIOM,
)

grecians_are_mortals = Implies(grecian(x), mortal(x), name="grecians_are_mortals")

model.add_formulae(grecians_are_mortals, all_grecians_are_native_or_resident)


Add facts provided by domain descripton. Note the difference on rows with `Zeus`. we use `Fact.UNKNOWN` to state that this fact is unknown.

In [16]:
from lnn import Fact

model.add_facts(
    {
        "native_of_greece": {
            "Great Pyramid of Giza": Fact.FALSE,
            "Confucius": Fact.FALSE,
            "Zeus": Fact.UNKNOWN,
            "Socrates": Fact.TRUE,
        },
        "resident_of_greece": {
            "Great Pyramid of Giza": Fact.FALSE,
            "Confucius": Fact.FALSE,
            "Zeus": Fact.UNKNOWN,
            "Socrates": Fact.FALSE,
        },
        "mortal": {
            "Great Pyramid of Giza": Fact.FALSE,
            "Confucius": Fact.TRUE,
            "Zeus": Fact.FALSE,
            "Socrates": Fact.TRUE,
        },
    }
)

### Inference inside LNN model

Now we run two inference passes, one for upward and one for downward inference.

In [17]:
from lnn import Direction

steps, facts_inferred = model.infer(direction=Direction.UPWARD)
steps, facts_inferred = model.infer(direction=Direction.DOWNWARD)

Expected result of inference is visible in this table:

![grecian_tt](images/grecian_uncertainty_tt_infered.png)

These results can be verified in following cell:

In [18]:
model["grecian"].print()

OPEN   Predicate: grecian(x0) 
'Confucius'                                                FALSE (0.0, 0.0)
'Socrates'                                                  TRUE (1.0, 1.0)
'Great Pyramid of Giza'                                    FALSE (0.0, 0.0)
'Zeus'                                                   UNKNOWN (0.0, 1.0)

